In [ ]:
from torch import nn
import torch
from torch.autograd import Variable

In [ ]:
class MCC_Loss(nn.Module):
    """
    Calculates the proposed Matthews Correlation Coefficient-based loss.
    Args:
        inputs (torch.Tensor): 1-hot encoded predictions
        targets (torch.Tensor): 1-hot encoded ground truth
    """

    def __init__(self):
        super(MCC_Loss, self).__init__()

    def forward(self, inputs, targets):
        """
        MCC = (TP.TN - FP.FN) / sqrt((TP+FP) . (TP+FN) . (TN+FP) . (TN+FN))
        where TP, TN, FP, and FN are elements in the confusion matrix.
        """
        tp = torch.sum(torch.mul(inputs, targets))
        tn = torch.sum(torch.mul((1 - inputs), (1 - targets)))
        fp = torch.sum(torch.mul(inputs, (1 - targets)))
        fn = torch.sum(torch.mul((1 - inputs), targets))

        numerator = torch.mul(tp, tn) - torch.mul(fp, fn)
        denominator = torch.sqrt(
            torch.add(tp, 1, fp)
            * torch.add(tp, 1, fn)
            * torch.add(tn, 1, fp)
            * torch.add(tn, 1, fn)
        )
        print("numerator", numerator)
        print("denominator", denominator)

        # Adding 1 to the denominator to avoid divide-by-zero errors.
        mcc = torch.div(numerator.sum(), denominator.sum() + 1.0)
        print("mcc", mcc)
        return 1 - mcc

In [ ]:
y = torch.tensor([1.,0.,1.,0.,1.,0.,1.,0.])
y_pred = torch.tensor([1.,1.,1.,1.,0.,0.,0.,0.])

In [ ]:
criterion = nn.BCELoss()

In [ ]:
loss = criterion(y_pred, y)

In [ ]:
loss

In [ ]:
loss.backward()